---

## 11.5 Beyond text classification: Sequence-to-sequence learning

TensorFlow [tutorial](https://www.tensorflow.org/text/tutorials/nmt_with_attention).

### 11.5.1 A machine translation example

### Download the dataset

```bash
!wget http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip
!unzip -q spa-eng.zip
```

In [10]:
import  sys, os
import re, string, random, pathlib
import numpy as np

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
# reminder: Colab code to mount your drive
if 'google.colab' in sys.modules:
    from google.colab import drive
    drive.mount('/content/drive')
    os.chdir('drive/My Drive/IS53024B-Artificial-Intelligence/6-text-and-sequences')      # 'My Drive' is the default name of Google Drives,
    os.listdir() 

Other languages available [here](https://www.manythings.org/anki/)! 

In [8]:
base_dir = pathlib.Path("spa-eng")
text_file = base_dir / "spa.txt"
 
with open(text_file) as f:
    lines = f.read().strip().split("\n")

text_pairs = []
separator = "\t" if "\t" in lines[0] else " " # spa.txt is tab separated, but the other datasets use spaces

for i, line in enumerate(lines):
    print(f"line: {i}/{len(lines)}\r", end="")
    english, spanish = line.split(separator)[:2] # for datasets other than the official one, only tak
    spanish = "[start] " + spanish + " [end]"
    text_pairs.append((english, spanish))

In [9]:
print(random.choice(text_pairs))

('She likes him.', '[start] A ella le gusta él. [end]')


In [7]:
random.shuffle(text_pairs)
num_val_samples = int(0.15 * len(text_pairs))
num_train_samples = len(text_pairs) - 2 * num_val_samples
train_pairs = text_pairs[:num_train_samples]
val_pairs = text_pairs[num_train_samples:num_train_samples + num_val_samples]
test_pairs = text_pairs[num_train_samples + num_val_samples:]

#### Vectorizing the English and Spanish text pairs

In [8]:
strip_chars = string.punctuation + "¿"
strip_chars = strip_chars.replace("[", "")
strip_chars = strip_chars.replace("]", "")

def custom_standardization(input_string):
    lowercase = tf.strings.lower(input_string)
    return tf.strings.regex_replace(
        lowercase, f"[{re.escape(strip_chars)}]", "")

vocab_size = 15000
sequence_length = 20

source_vectorization = layers.TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length,
)
target_vectorization = layers.TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length + 1,
    standardize=custom_standardization,
)
train_english_texts = [pair[0] for pair in train_pairs]
train_spanish_texts = [pair[1] for pair in train_pairs]
source_vectorization.adapt(train_english_texts)
target_vectorization.adapt(train_spanish_texts)

#### Preparing datasets for the translation task

In [23]:
batch_size = 1560

def format_dataset(eng, spa):
    eng = source_vectorization(eng)
    spa = target_vectorization(spa)
    return ({
        "english": eng,
        "spanish": spa[:, :-1],
    }, spa[:, 1:])

def make_dataset(pairs):
    eng_texts, spa_texts = zip(*pairs)
    eng_texts = list(eng_texts)
    spa_texts = list(spa_texts)
    dataset = tf.data.Dataset.from_tensor_slices((eng_texts, spa_texts))
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(format_dataset, num_parallel_calls=4)
    return dataset.shuffle(2048).prefetch(16).cache()

train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)

In [10]:
for inputs, targets in train_ds.take(1):
    print(f"inputs['english'].shape: {inputs['english'].shape}")
    print(f"inputs['spanish'].shape: {inputs['spanish'].shape}")
    print(f"targets.shape: {targets.shape}")

inputs['english'].shape: (64, 20)
inputs['spanish'].shape: (64, 20)
targets.shape: (64, 20)


---

### 11.5.2 Sequence-to-sequence learning with RNNs

#### GRU-based encoder

In [ ]:
embed_dim = 256
latent_dim = 1024

source = keras.Input(shape=(None,), dtype="int64", name="english")
x = layers.Embedding(vocab_size, embed_dim, mask_zero=True)(source)
encoded_source = layers.Bidirectional(
    layers.GRU(latent_dim), merge_mode="sum")(x)

#### GRU-based decoder and the end-to-end model

In [ ]:
past_target = keras.Input(shape=(None,), dtype="int64", name="spanish")
x = layers.Embedding(vocab_size, embed_dim, mask_zero=True)(past_target)
decoder_gru = layers.GRU(latent_dim, return_sequences=True)
x = decoder_gru(x, initial_state=encoded_source)
x = layers.Dropout(0.5)(x)
target_next_step = layers.Dense(vocab_size, activation="softmax")(x)
seq2seq_rnn = keras.Model([source, past_target], target_next_step)

#### Training our recurrent sequence-to-sequence model

In [ ]:
seq2seq_rnn.compile(
    optimizer="rmsprop",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)
seq2seq_rnn.fit(train_ds, epochs=15, validation_data=val_ds)

Epoch 1/15


2022-11-01 16:13:41.924830: W tensorflow/core/common_runtime/forward_type_inference.cc:231] Type inference failed. This indicates an invalid graph that escaped type checking. Error message: INVALID_ARGUMENT: expected compatible input types, but input 1:
type_id: TFT_OPTIONAL
args {
  type_id: TFT_PRODUCT
  args {
    type_id: TFT_TENSOR
    args {
      type_id: TFT_LEGACY_VARIANT
    }
  }
}
 is neither a subtype nor a supertype of the combined inputs preceding it:
type_id: TFT_OPTIONAL
args {
  type_id: TFT_PRODUCT
  args {
    type_id: TFT_TENSOR
    args {
      type_id: TFT_FLOAT
    }
  }
}

	while inferring type of node 'cond_41/output/_17'


1302/1302 [==============================] - 192s 125ms/step - loss: 1.6352 - accuracy: 0.4178 - val_loss: 1.3110 - val_accuracy: 0.5056
Epoch 2/15
1302/1302 [==============================] - 157s 121ms/step - loss: 1.3122 - accuracy: 0.5282 - val_loss: 1.1527 - val_accuracy: 0.5678
Epoch 3/15
1302/1302 [==============================] - 159s 122ms/step - loss: 1.1724 - accuracy: 0.5772 - val_loss: 1.0746 - val_accuracy: 0.5993
Epoch 4/15
1302/1302 [==============================] - 157s 120ms/step - loss: 1.0812 - accuracy: 0.6089 - val_loss: 1.0401 - val_accuracy: 0.6174
Epoch 5/15
1302/1302 [==============================] - 157s 121ms/step - loss: 1.0361 - accuracy: 0.6331 - val_loss: 1.0252 - val_accuracy: 0.6281
Epoch 6/15
1302/1302 [==============================] - 158s 121ms/step - loss: 1.0069 - accuracy: 0.6507 - val_loss: 1.0211 - val_accuracy: 0.6322
Epoch 7/15
1302/1302 [==============================] - 158s 121ms/step - loss: 0.9873 - accuracy: 0.6643 - val_loss: 1.020

The [issue (warning)](https://github.com/keras-team/keras/issues/16978) doesn't seem to be resolved yet...

#### Translating new sentences with our RNN encoder and decoder

In [31]:
def decode_sequence(input_sentence):
    tokenized_input_sentence = source_vectorization([input_sentence])
    decoded_sentence = "[start]"
    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = target_vectorization([decoded_sentence])
        next_token_predictions = seq2seq_rnn.predict(
            [tokenized_input_sentence, tokenized_target_sentence], verbose=0)
        sampled_token_index = np.argmax(next_token_predictions[0, i, :])
        sampled_token = spa_index_lookup[sampled_token_index]
        decoded_sentence += " " + sampled_token
        if sampled_token == "[end]":
            break
    return decoded_sentence

In [30]:
spa_vocab = target_vectorization.get_vocabulary()
spa_index_lookup = dict(zip(range(len(spa_vocab)), spa_vocab))
max_decoded_sentence_length = 20
test_eng_texts = [pair[0] for pair in test_pairs]

In [ ]:
for _ in range(20):
    input_sentence = random.choice(test_eng_texts)
    print("-")
    print(input_sentence)
    print(decode_sequence(input_sentence))

-
It's up to you to decide.
[start] es usted a [UNK] [end]
-
The jury has returned a verdict of guilty.
[start] el [UNK] ha sido un [UNK] de lo que y el [UNK] [end]
-
Tom is devious, isn't he?
[start] tom es [UNK] verdad [end]
-
I don't know her real name.
[start] no sé su nombre de su nombre [end]
-
He shook hands with his friend.
[start] Él [UNK] las manos con su amigo [end]
-
She is a wealthy woman.
[start] ella es una mujer mujer [end]
-
Could you hand me the newspaper on the table?
[start] me puedes el libro sobre la mesa [end]
-
The train leaves at six.
[start] el tren las seis en punto [end]
-
Come again tomorrow.
[start] ven mañana [end]
-
I tried to talk a friend of mine out of getting married.
[start] intentó un amigo de las cosas que tom [end]
-
Can you see the difference?
[start] puedes ver la guitarra [end]
-
Anybody home?
[start] alguien está en casa [end]
-
Tom described how he was able to escape from the locked room.
[start] tom sabía que él fue capaz de ver la habitaci

---

### 11.5.3 Sequence-to-sequence learning with Transformer

#### Putting it all together: A Transformer for machine translation

<!-- <img style="" src="images/transformer/the-annotated-transformer_14_0.transformer-full.png"> -->
<img src="https://drive.google.com/uc?id=1q24jSyMvzKI160pR6VDmY30i7ulwHKIK">


<small>[Vaswani et al, "Attention Is All You Need"](https://arxiv.org/abs/1706.03762)</small>

#### The Transformer decoder

In [34]:
# use of tf.tile
a = tf.constant([[1,2,3],[4,5,6]], tf.int32)
b = tf.constant([1,2], tf.int32)
c = tf.constant([2,1], tf.int32)

print(a.numpy())
print(tf.tile(a,b).numpy())
print(tf.tile(a,c).numpy())

[[1 2 3]
 [4 5 6]]
[[1 2 3 1 2 3]
 [4 5 6 4 5 6]]
[[1 2 3]
 [4 5 6]
 [1 2 3]
 [4 5 6]]


In [36]:
def get_causal_attention_mask(inputs):
    print("Inputs:")
    print(inputs)
    print()
    input_shape = tf.shape(inputs)
    batch_size, sequence_length = input_shape[0], input_shape[1]
    i = tf.range(sequence_length)[:, tf.newaxis]
    j = tf.range(sequence_length)
    print(f"i:\n{i}")
    print()
    print(f"j:\n{j}")
    print()
    mask = tf.cast(i >= j, dtype="int32")
    print("Is i >= i? Boolean cast to ints. (Note the broadcasting)")
    print()
    print(mask)
    print()
    mask = tf.reshape(mask, (1, input_shape[1], input_shape[1])) # adding a batch dimension
    mult = tf.concat(
        [tf.expand_dims(batch_size, -1),
         tf.constant([1, 1], dtype=tf.int32)], axis=0)
    print("We want mask to have the same dims as input, using `tf.tile`.")
    print("Creating the right multiplier for it:")
    print()
    print(mult)
    print()
    tile = tf.tile(mask, mult)
    print("Final mask with batch dimensions:")
    print()
    print(tile)
    return tile

mask = get_causal_attention_mask(tf.random.uniform(shape=(2,10), maxval=50, dtype=tf.int32))

Inputs:
tf.Tensor(
[[15 20 16 20 47 19  6 23  8 31]
 [43 38 10  6 14 18 44 29 24 33]], shape=(2, 10), dtype=int32)

i:
[[0]
 [1]
 [2]
 [3]
 [4]
 [5]
 [6]
 [7]
 [8]
 [9]]

j:
[0 1 2 3 4 5 6 7 8 9]

Is i >= i? Boolean cast to ints. (Note the broadcasting)

tf.Tensor(
[[1 0 0 0 0 0 0 0 0 0]
 [1 1 0 0 0 0 0 0 0 0]
 [1 1 1 0 0 0 0 0 0 0]
 [1 1 1 1 0 0 0 0 0 0]
 [1 1 1 1 1 0 0 0 0 0]
 [1 1 1 1 1 1 0 0 0 0]
 [1 1 1 1 1 1 1 0 0 0]
 [1 1 1 1 1 1 1 1 0 0]
 [1 1 1 1 1 1 1 1 1 0]
 [1 1 1 1 1 1 1 1 1 1]], shape=(10, 10), dtype=int32)

We want mask to have the same dims as input, using `tf.tile`.
Creating the right multiplier for it:

tf.Tensor([2 1 1], shape=(3,), dtype=int32)

Final mask with batch dimensions:

tf.Tensor(
[[[1 0 0 0 0 0 0 0 0 0]
  [1 1 0 0 0 0 0 0 0 0]
  [1 1 1 0 0 0 0 0 0 0]
  [1 1 1 1 0 0 0 0 0 0]
  [1 1 1 1 1 0 0 0 0 0]
  [1 1 1 1 1 1 0 0 0 0]
  [1 1 1 1 1 1 1 0 0 0]
  [1 1 1 1 1 1 1 1 0 0]
  [1 1 1 1 1 1 1 1 1 0]
  [1 1 1 1 1 1 1 1 1 1]]

 [[1 0 0 0 0 0 0 0 0 0]
  [1 1 0 0 0 0

In [12]:
class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim                       # parameters
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = layers.MultiHeadAttention(      # multi-head attention layer
            num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential(              # dense layers at the top
            [layers.Dense(dense_dim, activation="relu"),
             layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()   # layer norm layers
        self.layernorm_2 = layers.LayerNormalization()

    def call(self, inputs, mask=None):
        if mask is not None:                             # mask used to enforce causality
            mask = mask[:, tf.newaxis, :]
        attention_output = self.attention(
            inputs, inputs, attention_mask=mask)
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)

    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config

In [13]:
class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim                     # parameters
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(  # multi-head attention layers
            num_heads=num_heads, key_dim=embed_dim)
        self.attention_2 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential(            # dense layers at the top
            [layers.Dense(dense_dim, activation="relu"),
             layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization() # layer norm layers
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        self.supports_masking = True                   # MASK: enforcing causality

    def get_config(self):                              # retrieve config as a dict
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config

    def get_causal_attention_mask(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = tf.range(sequence_length)[:, tf.newaxis]
        j = tf.range(sequence_length)
        mask = tf.cast(i >= j, dtype="int32")
        mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))
        mult = tf.concat(
            [tf.expand_dims(batch_size, -1),
             tf.constant([1, 1], dtype=tf.int32)], axis=0)
        return tf.tile(mask, mult)

    def call(self, inputs, encoder_outputs, mask=None):
        causal_mask = self.get_causal_attention_mask(inputs)
        if mask is not None:
            padding_mask = tf.cast(
                mask[:, tf.newaxis, :], dtype="int32")
            padding_mask = tf.minimum(padding_mask, causal_mask)
        attention_output_1 = self.attention_1(
            query=inputs,
            value=inputs,
            key=inputs,
            attention_mask=causal_mask)                # first layer: apply the causal mask
        attention_output_1 = self.layernorm_1(inputs + attention_output_1)
        attention_output_2 = self.attention_2(
            query=attention_output_1,
            value=encoder_outputs,
            key=encoder_outputs,
            attention_mask=padding_mask,
        )
        attention_output_2 = self.layernorm_2(
            attention_output_1 + attention_output_2)
        proj_output = self.dense_proj(attention_output_2)
        return self.layernorm_3(attention_output_2 + proj_output)

#### PositionalEmbedding layer

In [14]:
class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, input_dim, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.token_embeddings = layers.Embedding(
            input_dim=input_dim, output_dim=output_dim)
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=output_dim)
        self.sequence_length = sequence_length
        self.input_dim = input_dim
        self.output_dim = output_dim

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)

    def get_config(self):
        config = super(PositionalEmbedding, self).get_config()
        config.update({
            "output_dim": self.output_dim,
            "sequence_length": self.sequence_length,
            "input_dim": self.input_dim,
        })
        return config

#### End-to-end Transformer

In [15]:
embed_dim = 256
dense_dim = 2048
num_heads = 8

encoder_inputs = keras.Input(shape=(None,), dtype="int64", name="english")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(encoder_inputs)
encoder_outputs = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)

decoder_inputs = keras.Input(shape=(None,), dtype="int64", name="spanish")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(decoder_inputs)
x = TransformerDecoder(embed_dim, dense_dim, num_heads)(x, encoder_outputs)
x = layers.Dropout(0.5)(x)
decoder_outputs = layers.Dense(vocab_size, activation="softmax")(x)
transformer = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

#### Training the sequence-to-sequence Transformer

In [37]:
transformer.compile(
    optimizer="rmsprop",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)
transformer.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 english (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 spanish (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 positional_embedding (Position  (None, None, 256)   3845120     ['english[0][0]']                
 alEmbedding)                                                                                     
                                                                                                  
 positional_embedding_1 (Positi  (None, None, 256)   3845120     ['spanish[0][0]']            

In [24]:
transformer.fit(train_ds, epochs=30, validation_data=val_ds)

Epoch 1/30
54/54 [==============================] - 65s 1s/step - loss: 1.5495 - accuracy: 0.4139 - val_loss: 1.3521 - val_accuracy: 0.4445
Epoch 2/30
54/54 [==============================] - 60s 1s/step - loss: 1.3139 - accuracy: 0.4766 - val_loss: 1.1480 - val_accuracy: 0.5164
Epoch 3/30
54/54 [==============================] - 60s 1s/step - loss: 1.1538 - accuracy: 0.5275 - val_loss: 1.0796 - val_accuracy: 0.5381
Epoch 4/30
54/54 [==============================] - 60s 1s/step - loss: 1.0309 - accuracy: 0.5706 - val_loss: 0.9525 - val_accuracy: 0.5922
Epoch 5/30
54/54 [==============================] - 60s 1s/step - loss: 0.9187 - accuracy: 0.6093 - val_loss: 0.8776 - val_accuracy: 0.6191
Epoch 6/30
54/54 [==============================] - 60s 1s/step - loss: 0.8293 - accuracy: 0.6414 - val_loss: 0.8200 - val_accuracy: 0.6437
Epoch 7/30
54/54 [==============================] - 60s 1s/step - loss: 0.7553 - accuracy: 0.6669 - val_loss: 0.7993 - val_accuracy: 0.6439
Epoch 8/30
54/54 [==

#### Translating new sentences with our Transformer model

In [33]:
def decode_sequence(input_sentence):
    tokenized_input_sentence = source_vectorization([input_sentence])
    decoded_sentence = "[start]"
    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = target_vectorization(
            [decoded_sentence])[:, :-1]
        predictions = transformer(
            [tokenized_input_sentence, tokenized_target_sentence])
        sampled_token_index = np.argmax(predictions[0, i, :])
        sampled_token = spa_index_lookup[sampled_token_index]
        decoded_sentence += " " + sampled_token
        if sampled_token == "[end]":
            break
    return decoded_sentence

In [34]:
spa_vocab = target_vectorization.get_vocabulary()
spa_index_lookup = dict(zip(range(len(spa_vocab)), spa_vocab))
max_decoded_sentence_length = 20
test_eng_texts = [pair[0] for pair in test_pairs]

In [36]:
for _ in range(20):
    input_sentence = random.choice(test_eng_texts)
    print("-")
    print(input_sentence)
    print(decode_sequence(input_sentence))

-
How is it that you can speak this language?
[start] cómo es que hablar en este idioma [end]
-
I just want to take a quick look.
[start] solo quiero tomar una mirada [end]
-
You should read more.
[start] deberías leer más [end]
-
Have you been told when to come?
[start] te has dicho cuándo venga [end]
-
Are you for or against the war?
[start] estás a favor o en la guerra [end]
-
You do not realize how important health is until you get sick.
[start] no te das cuenta de lo que es importante que te das enfermo [end]
-
Crossing that desert is dangerous.
[start] la gente que es peligroso el mundo sabe peligroso [end]
-
Do people ever accuse you of being snobbish?
[start] la gente de la gente se hace a la gente que te [UNK] [end]
-
Tom tried to rescue Mary.
[start] tom intentó salvar a mary [end]
-
I'll always remember you.
[start] siempre te amaré [end]
-
I grabbed my little sister's hand and started running.
[start] cogí mi hermana pequeña hermana y salió a correr [end]
-
Tom told us all 

## Summary